## Opening the file

In [11]:
import pandas as pd 
data = "C:\\Users\\varsh\\Downloads\\Arts_Crafts_and_sewing_5.csv"
# reading the fie and print
df = pd.read_csv(data)
df.sample(10)

,overall,verified,reviewerID,asin,reviewText,summary
449478,4,True,A3CS2KV9LXO1H,B006PGEPM2,Works rather well for a commercial version of ...,Works fine
225741,5,True,A17LLJBKXPHDHP,B0053TN3F2,This is a great set to use and the box is well...,This is a wonderful set to work with
491735,5,True,AYJV5ASYGCKI9,B01BMQ33PE,I have purchased a lot of stabilizer in my day...,Feels so soft but strong
424197,3,True,A3MMJ6C99ZTGG9,B000YZ8N46,Just received and have used once already. Seem...,Three Stars
140061,5,True,A1BC0UNJZTM3GZ,B001C26RFM,good stuff,Five Stars
396072,5,False,A23I74FX8I99S5,B018INJ8U8,Blades function excellently!,Five Stars
238600,5,True,A2CUE2FCSTRIG2,B005RE4AC8,I love this string. It is strong and braids wi...,Great to have different colors
171557,3,True,A3KZB0DQFYYA8Z,B0027AAIB2,tracing paper is tracing paper. it is hand fo...,Better in the pad than loose leafs
232533,3,True,A1HSJNQ2RTOG2F,B005G890UM,some times i have to rub like crazy to get it ...,Three Stars
233057,5,True,A36FVWJ2SZQ6NH,B005GYJZMY,Bought for daughter. She loves all the colors.,Nice colors selection


## Getting the Bing Liu Lexicon

In [12]:
from sklearn import preprocessing
import nltk# Import the Natural Language Toolkit library
nltk.download('opinion_lexicon')
# Load positive and negative words from the opinion lexicon
from nltk.corpus import opinion_lexicon
from nltk.tokenize import word_tokenize

[nltk_data] Downloading package opinion_lexicon to
[nltk_data]     C:\Users\varsh\AppData\Roaming\nltk_data...
[nltk_data]   Package opinion_lexicon is already up-to-date!


In [13]:
# Print the total number of words in the opinion lexicon
print('Total number of words in opinion lexicon', len(opinion_lexicon.words()))
print('Examples of positive words in opinion lexicon',
opinion_lexicon.positive()[:10])
print('Examples of negative words in opinion lexicon',
opinion_lexicon.negative()[:10])

Total number of words in opinion lexicon 6789
Examples of positive words in opinion lexicon ['a+', 'abound', 'abounds', 'abundance', 'abundant', 'accessable', 'accessible', 'acclaim', 'acclaimed', 'acclamation']
Examples of negative words in opinion lexicon ['2-faced', '2-faces', 'abnormal', 'abolish', 'abominable', 'abominably', 'abominate', 'abomination', 'abort', 'aborted']


In [14]:
# Let's create a dictionary which we can use for scoring our review text
nltk.download('punkt')
df.rename(columns={"reviewText": "text"}, inplace=True)
pos_score = 1
neg_score = -1
word_dict = {}


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\varsh\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [15]:
# Adding the positive words to the dictionary
for word in opinion_lexicon.positive():
 word_dict[word] = pos_score
# Adding the negative words to the dictionary
for word in opinion_lexicon.negative():
 word_dict[word] = neg_score

## Function to do math

In [24]:
def bing_liu_score(text):
    sentiment_score = 0
    bag_of_words = word_tokenize(text.lower())
    for word in bag_of_words:
        if word in word_dict:
            sentiment_score += word_dict[word]
    return sentiment_score

## Making sure all texts are non empty and are type text then call the function

In [25]:
# Fill missing values in the 'text' column with 'no review'
df['text'].fillna('no review', inplace=True)
df['Bing_Liu_Score'] = df['text'].apply(bing_liu_score)


In [26]:
df[['overall',"text", 'Bing_Liu_Score']].head(10)

,overall,text,Bing_Liu_Score
0,4,Contains some interesting stitches.,1
1,5,I'm a fairly experienced knitter of the one-co...,22
2,4,Great book but the index is terrible. Had to w...,0
3,5,I purchased the Kindle edition which is incred...,4
4,5,Very well laid out and very easy to read.\n\nT...,5
5,5,"Beginning her career as a freelance knitter, M...",15
6,5,This is a terrific stitch handbook (and I have...,9
7,4,The book needs to be coil bound. The content i...,1
8,5,I really am enjoying this book! I like the siz...,12
9,5,Just received this book and looked over it cov...,6


In [27]:
df.groupby('overall').agg({'Bing_Liu_Score':'mean'})

,Bing_Liu_Score
overall,
1,-0.255049
2,0.566098
3,1.158796
4,2.027999
5,2.129986
